In [1]:
"""EchoSeed v3.20 – Watermark Initialisation.
Placed automatically at top of notebook to embed a session fingerprint in every glyph.
"""

# >>> Watermark Initialisation (first-cell block) <<<
import hashlib, time
SESSION_HASH = hashlib.sha256(f"EchoSeed_v3.20_{int(time.time())}".encode()).hexdigest()[:8]  # eight-char fingerprint unique per runtime

In [2]:
# --- PATCH: make json.dump handle numpy arrays automatically ---
import numpy as np, json

def _np_default(o):
    if isinstance(o, np.ndarray):
        return o.tolist()
    if isinstance(o, np.generic):  # e.g., np.int32, np.float64
        return o.item()
    raise TypeError(f'{type(o).__name__} not JSON serialisable')

_json_dump_original = json.dump

def _json_dump_np(obj, fp, *args, default=None, **kwargs):
    if default is None:
        default = _np_default
    return _json_dump_original(obj, fp, *args, default=default, **kwargs)

json.dump = _json_dump_np
# --- END PATCH ---

# 🌱 EchoSeed v3.0
Clean rebuild with:
- Drive persistence (chunks + master JSON)
- Single-thread guard + rate slider
- Lattice render capped at 1000 nodes
- Entropy graph
- No indentation headaches 🚀

In [3]:

!pip -q install sentence-transformers
from sentence_transformers import SentenceTransformer
# 📦 Imports
import random, json, threading, time, os
from IPython.display import display, clear_output
import ipywidgets as widgets
import networkx as nx
import numpy as np
from scipy.spatial.distance import cosine
import collections
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.7 MB/s eta 0:00:00


In [4]:

# 🔠 Load sentence-transformers model for semantic embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:

# 📂 Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/EchoSeed_Logs'
os.makedirs(drive_path, exist_ok=True)

master_log_path = os.path.join(drive_path, 'master_log.json')
chunk_index_path = os.path.join(drive_path, 'chunk_index.txt')
if not os.path.exists(chunk_index_path):
    with open(chunk_index_path, 'w') as f:
        f.write('0')

Mounted at /content/drive


In [7]:
# 🔧 Glyph Engine (v3.9 – stable parent lookup)
glyph_log=[]
id2glyph={}
glyph_graph=nx.Graph()
graph_lock=threading.Lock()
glyph_id=0
running=threading.Event()
seen_signatures=set()
reflex_free=[]
stagnant_counter={}
MAX_RAM_GLYPHS=20_000
LOW_ENTROPY=100
LOW_KEEP_PROB=0.025   # 2.5% of low-entropy glyphs
PRUNE_AFTER=250


# ---- Semantic embedding helpers (Sentence-Transformers) ----
def tag_vec(tag):
    return model.encode(tag)

def embed(tags):
    return model.encode(", ".join(sorted(tags)))

def semantic_novelty(vec):
    if not glyph_log:
        return 1.0
    sample = random.sample(glyph_log, min(100, len(glyph_log)))
    dists = [cosine(vec, g['vec']) for g in sample]
    return max(dists)


def embed(tags):
    vs = [tag_vec(t) for t in tags]
    return np.mean(vs, axis=0)

def semantic_novelty(vec):
    if not glyph_log:
        return 1.0
    # sample 100 random glyphs for speed
    sample = random.sample(glyph_log, min(100, len(glyph_log)))
    dists = [cosine(vec, g['vec']) for g in sample]
    return max(dists)  # higher distance -> novel


def gen_id():
    global glyph_id
    glyph_id+=1
    return f"g{glyph_id:04}"

def random_tag():
    return random.choice(['origin','flex','ghost','fractal','wild','mirror','unknown','stable','reflex'])

def calc_entropy(g):
    return len(g['tags'])*42+random.randint(0,58)


def fitness(g):
    sem = semantic_novelty(g['vec'])
    return 200*sem - g['entropy']


def mutate_tag(tags):
    if len(tags) < 2:
        return 'm_misc'
    a,b = random.sample(tags,2)
    return f"{a}+{b}"

def signature(g,bucket=25):
    return (tuple(sorted(g['tags'])),g['entropy']//bucket)

def is_novel(g):
    if g['entropy']<LOW_ENTROPY:
        return random.random()<LOW_KEEP_PROB
    sig=signature(g)
    if sig in seen_signatures:
        return False
    seen_signatures.add(sig)
    return True

def maybe_store(g):
    g['vec'] = embed(g['tags'])  # attach semantic vector
    if is_novel(g):
        glyph_log.append(g)
        id2glyph[g['id']]=g
        with graph_lock:
            glyph_graph.add_node(g['id'],entropy=g['entropy'],tags=g['tags'],fit=fitness(g))
            stagnant_counter[g['id']]=0
    if 'reflex' not in g['tags']:
        reflex_free.append(g)

def create_glyph():
    g={'id':gen_id(),
       'tags':list({random_tag() for _ in range(random.randint(1,3))}),
       'entropy':0,
       'ancestry':[]}
    g['entropy']=calc_entropy(g)
    g['vec'] = embed(g['tags'])
    return g


def collide(a, b):
    child_tags = {*a['tags'], *b['tags'], mutate_tag(a['tags'] + b['tags'])}
    child = {'id': gen_id(), 'tags': list(child_tags), 'entropy': 0, 'ancestry': [a['id'], b['id']]}
    child['entropy'] = calc_entropy(child)
    maybe_store(child)
    with graph_lock:
        glyph_graph.add_edge(a['id'], child['id'])
        glyph_graph.add_edge(b['id'], child['id'])
        stagnant_counter[a['id']] = stagnant_counter[b['id']] = stagnant_counter[child['id']] = 0
        # Greedy edge explosion
        candidates = [g for g in glyph_log if g['id'] not in {a['id'], b['id'], child['id']}]
        for _ in range(min(20, len(candidates))):  # 20 edges max
            extra = random.choice(candidates)
            glyph_graph.add_edge(child['id'], extra['id'])
            stagnant_counter[extra['id']] = 0

def select_parents(k=10):
    with graph_lock:
        nodes=list(glyph_graph.nodes(data=True))
    if len(nodes)<2:
        return []
    weights=[max(1,nd.get('fit',1)) for _,nd in nodes]
    chosen=random.choices(nodes,weights=weights,k=min(k,len(nodes)))
    return [n[0] for n in chosen]

def batch_collide(max_pairs=30):
    parents=select_parents(max_pairs)
    if len(parents)<2:
        return
    for i in range(len(parents)):
        for j in range(i+1,len(parents)):
            a=id2glyph.get(parents[i])
            b=id2glyph.get(parents[j])
            if a and b:
                collide(a,b)

def reflex_test():
    for g in [g for g in glyph_log if 'unknown' in g['tags'] and g['entropy'] < 150]:
        for _ in range(3):  # Triple the offspring
            new_g = create_glyph()
            new_g['tags'].append('reflex')
            maybe_store(new_g)
            with graph_lock:
                glyph_graph.add_edge(g['id'], new_g['id'])
            collide(g, new_g)

def prune_stagnant():
    to_remove=[]
    with graph_lock:
        for node in list(glyph_graph.nodes):
            stagnant_counter[node]+=1
            if stagnant_counter[node]>PRUNE_AFTER:
                to_remove.append(node)
        for n in to_remove:
            glyph_graph.remove_node(n)
            stagnant_counter.pop(n,None)
    for nid in to_remove:
        id2glyph.pop(nid,None)
    ids=set(to_remove)
    glyph_log[:]=[g for g in glyph_log if g['id'] not in ids]



def rewire_shortcuts(pct=0.05):
    """Add random shortcut edges (pct of nodes) each save."""
    with graph_lock:
        nodes = list(glyph_graph.nodes)
    if len(nodes) < 3:
        return
    sample = random.sample(nodes, max(2, int(len(nodes) * pct)))
    for nid in sample:
        target = random.choice(nodes)
        if target != nid:
            with graph_lock:
                glyph_graph.add_edge(nid, target)
                stagnant_counter[nid] = 0
                stagnant_counter[target] = 0

def save_logs(chunk_size=1000):
    prune_stagnant()
    rewire_shortcuts()
    # RAM cap
    if len(glyph_log) > MAX_RAM_GLYPHS:
        del glyph_log[:-MAX_RAM_GLYPHS]

    with open(chunk_index_path, 'r') as f:
        idx = int(f.read().strip())
    chunk = glyph_log[-chunk_size:]

    with open(os.path.join(drive_path, f'chunk_{idx:04}.json'), 'w', encoding='utf-8') as f:
        json.dump(chunk, f, indent=2, ensure_ascii=False)

    with open(chunk_index_path, 'w') as f:
        f.write(str(idx + 1))

    try:
        with open(master_log_path, 'r', encoding='utf-8') as f:
            master = json.load(f)
    except FileNotFoundError:
        master = []
    master += chunk
    with open(master_log_path, 'w', encoding='utf-8') as f:
        json.dump(master, f, indent=2, ensure_ascii=False)

    with open(os.path.join(drive_path, 'reflex_free.json'), 'w', encoding='utf-8') as f:
        json.dump(reflex_free, f, indent=2, ensure_ascii=False)

generation_count = 0
def calc_entropy(g, gen=generation_count):
    return len(g['tags']) * 42 + random.randint(0, 58) + (gen * 10)  # Entropy grows with generations
def generate():
    global generation_count
    while running.is_set():
        generation_count += 1
        g = create_glyph()
        maybe_store(g)
        batch_collide()
        reflex_test()
        if len(glyph_log) % 250 == 0 and len(glyph_log) > 0:
            save_logs()
        time.sleep(gen_rate_slider.value)

# snapshot cluster analysis
if len(glyph_log) >= 500:
    vec_sample = np.stack([g['vec'] for g in glyph_log])
    mb = MiniBatchKMeans(n_clusters=20, batch_size=256).fit(vec_sample)
    cluster_counts = collections.Counter(mb.labels_.tolist())
    with open(os.path.join(drive_path,'cluster_counts.json'),'w',encoding='utf-8') as cf:
        json.dump(cluster_counts, cf, indent=2)

In [8]:

# 🎛️ UI
output_area = widgets.Output()
entropy_output = widgets.Output()
gen_rate_slider = widgets.FloatSlider(value=0.5,min=0.1,max=5.0,step=0.1,description='Rate (s)',readout_format='.1f')

def start_gen(_):
    if running.is_set():
        return
    running.set()
    threading.Thread(target=generate, daemon=True).start()

def stop_gen(_):
    running.clear()

def render_lattice_graph():
    with graph_lock:
        recent_ids = [g['id'] for g in glyph_log][-1000:]
        subG = glyph_graph.subgraph(recent_ids).copy()
    if not subG.nodes:
        print('No glyphs yet.')
        return
    pos = nx.kamada_kawai_layout(subG)
    ents = [subG.nodes[n].get('entropy', 0) for n in subG.nodes]
    nx.draw(subG, pos, nodelist=list(subG.nodes), node_color=ents, node_size=100,
            edgelist=subG.edges, cmap=plt.cm.plasma, with_labels=False)
    plt.title(f'Lattice View (last {len(subG.nodes)} glyphs)')
    plt.show()
def render_lattice_ui(_):
    with output_area:
        clear_output()
        render_lattice_graph()

def render_entropy_graph(_=None):
    with entropy_output:
        clear_output()
        if not glyph_log:
            print("No data.")
            return
        plt.figure(figsize=(10,3))
        ents=[g['entropy'] for g in glyph_log]
        plt.plot(ents,marker='.',linewidth=1)
        plt.title("Entropy over time")
        plt.xlabel("Glyph index")
        plt.ylabel("Entropy")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# Buttons
start_btn=widgets.Button(description="Start")
stop_btn=widgets.Button(description="Stop")
lattice_btn=widgets.Button(description="Render Lattice")
entropy_btn=widgets.Button(description="Entropy Graph")
save_btn = widgets.Button(description='Save Now')
reflex_btn = widgets.Button(description='Reflex‑Free Count')

start_btn.on_click(start_gen)
stop_btn.on_click(stop_gen)
lattice_btn.on_click(render_lattice_ui)
entropy_btn.on_click(render_entropy_graph)
save_btn.on_click(lambda _: save_logs())
reflex_btn.on_click(lambda _ : print(f'Reflex‑free glyphs → {len(reflex_free)}'))

display(widgets.HBox([start_btn, stop_btn, lattice_btn, entropy_btn, gen_rate_slider, save_btn, reflex_btn]))
display(output_area)
display(entropy_output)

Output()

Output()

Reflex‑free glyphs → 916


In [ ]:
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# ---- Step 1: Load your glyph log (adjust path if needed) ----
with open("/content/drive/MyDrive/EchoSeed_Logs/master_log.json", "r") as f:
    glyph_log = json.load(f)

# ---- Step 2: Define helpers for the lattice ----
def build_lattice_graph(glyph_log):
    G = nx.DiGraph()
    for glyph in glyph_log:
        gid = glyph["id"]
        G.add_node(gid,
                   entropy=glyph.get("entropy", 0),
                   tags=glyph.get("tags", []),
                   ancestry=glyph.get("ancestry", []))
    for glyph in glyph_log:
        gid = glyph["id"]
        ancestors = glyph.get("ancestry", [])
        for ancestor in ancestors:
            if ancestor in G.nodes:
                G.add_edge(ancestor, gid)
    return G

def calculate_node_levels(G):
    levels = {}
    root_nodes = [n for n in G.nodes if G.in_degree(n) == 0]
    queue = [(node, 0) for node in root_nodes]
    while queue:
        node, level = queue.pop(0)
        if node not in levels or level > levels[node]:
            levels[node] = level
            for successor in G.successors(node):
                queue.append((successor, level + 1))
    return levels

def get_entropy_array(G):
    entropies = np.array([G.nodes[n].get("entropy", 0) for n in G.nodes])
    if len(entropies) == 0:
        return np.ones(len(G.nodes)) * 100
    entropies = np.clip(entropies, 0, None)
    if np.max(entropies) > 0:
        norm = entropies / np.max(entropies)
    else:
        norm = np.ones_like(entropies)
    return norm * 400 + 100

def create_hierarchical_layout(G, levels):
    pos = {}
    level_nodes = {}
    for node, level in levels.items():
        if level not in level_nodes:
            level_nodes[level] = []
        level_nodes[level].append(node)
    max_level = max(levels.values()) if levels else 0
    for level, nodes in level_nodes.items():
        y = max_level - level
        for i, node in enumerate(nodes):
            x = i - (len(nodes) - 1) / 2
            pos[node] = (x, y)
    return pos

# ---- Step 3: Your watermarked render function ----
def render_enhanced_lattice_watermarked(G, layout_type="hierarchical"):
    levels = calculate_node_levels(G)
    node_sizes = get_entropy_array(G)
    if layout_type == "hierarchical":
        pos = create_hierarchical_layout(G, levels)
    else:
        pos = nx.spring_layout(G, seed=42, k=2, iterations=50)
    fig, ax = plt.subplots(figsize=(16, 12))
    node_colors = []
    for node in G.nodes:
        tags = G.nodes[node].get("tags", [])
        if "origin" in tags:
            node_colors.append("#ff6b6b")
        elif G.in_degree(node) == 0:
            node_colors.append("#4ecdc4")
        else:
            entropy = G.nodes[node].get("entropy", 0)
            if entropy > 0.5:
                node_colors.append("#ffe66d")
            else:
                node_colors.append("#a8e6cf")
    nx.draw_networkx_edges(G, pos, edge_color="#2c3e50", width=2, alpha=0.7, arrows=True, arrowsize=20)
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.8, linewidths=2, edgecolors="#2c3e50")
    nx.draw_networkx_labels(G, pos, font_size=8, font_weight="bold")
    ax.set_title("EchoSeed Recursive Lattice Structure", fontsize=16, fontweight="bold", pad=20)
    legend_elements = [
        mpatches.Patch(color="#ff6b6b", label="Origin Nodes"),
        mpatches.Patch(color="#4ecdc4", label="Root Nodes"),
        mpatches.Patch(color="#ffe66d", label="High Entropy"),
        mpatches.Patch(color="#a8e6cf", label="Low Entropy"),
        plt.Line2D([0], [0], color="#2c3e50", lw=2, label="Hierarchical")
    ]
    ax.legend(handles=legend_elements, loc="upper right", bbox_to_anchor=(1.15, 1))
    info_text = f"Nodes: {len(G.nodes)}\nEdges: {len(G.edges)}\n"
    info_text += f"Max Level: {max(levels.values()) if levels else 0}\n"
    info_text += f"Root Nodes: {len([n for n in G.nodes if G.in_degree(n) == 0])}"
    ax.text(0.02, 0.98, info_text, transform=ax.transAxes,
            bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8),
            verticalalignment="top", fontsize=10)
    # Add semi-transparent watermark
    ax.text(
        0.98, 0.01,
        "EchoSeed v3.21 · @Duhmeee · 2025-07-10",
        fontsize=16, color="#888888", alpha=0.38,
        ha="right", va="bottom",
        transform=ax.transAxes,
        fontweight="bold", fontname="DejaVu Sans"
    )
    plt.tight_layout()
    plt.show()

# ---- Step 4: Build the graph object G and render ----
G = build_lattice_graph(glyph_log)
render_enhanced_lattice_watermarked(G)

In [ ]:
# --- EXPORT 10k GLYPH OBJECTS -----------------------------------
import json, random

#  ⬇️  point this to your full glyph log list
raw = glyph_log                                  # already in memory

# Choose a 1 000-glyph sample so the file stays light
sample = random.sample(raw, k=min(10000, len(raw)))

TAG_VOCAB = sorted({"mirror", "ghost", "reflex", "flex", "unknown"})
MAX_ENT   = max(g["entropy"] for g in sample) or 1.0   # normalise

def one_hot(tags):
    return [1 if t in tags else 0 for t in TAG_VOCAB]

objects = []
for g in sample:
    feature = one_hot(g["tags"])
    feature.append(g["entropy"] / MAX_ENT)      # scaled entropy
    objects.append({"id": g["id"],
                    "feat": feature,
                    "tags": g["tags"]})

with open("objects.json", "w") as f:
    json.dump(objects, f, indent=2)

print("✓  wrote", len(objects), "objects to objects.json")

In [ ]:
# --- EXPORT RESONANCE EDGES (quick heuristic) ------------------
from itertools import combinations
import json, collections

tag_sets = {o["id"]: set(o["tags"]) for o in objects}

edges = []
for (u, v) in combinations(tag_sets, 2):
    # simple overlap score = number of shared tags
    overlap = len(tag_sets[u].intersection(tag_sets[v]))
    if overlap >= 2:                       # a crude “resonant” edge
        edges.append({"subj": u, "obj": v, "type": "resonant"})
    else:
        continue   # skip non-resonant edges for this demo

with open("relations.json", "w") as f:
    json.dump(edges, f, indent=2)

print("✓  wrote", len(edges), "edges to relations.json")

In [ ]:
%%writefile echo_nscl_demo.py
# echo_nscl_demo.py  – 10-line demo that proves the plumbing
import json, torch, torch.nn as nn, torch.optim as optim, random, sys, os
from pathlib import Path

# ---------- tiny loader -------------------------------------------------
def load_objs(path="objects.json"):
    objs = json.load(open(path))
    X = torch.tensor([o["feat"] for o in objs], dtype=torch.float32)
    # label = 1 if any obj has *both* mirror & ghost tags
    y = torch.tensor([[int("mirror" in o["tags"] and "ghost" in o["tags"])]
                      for o in objs], dtype=torch.float32)
    return X, y

X, y = load_objs()

# ---------- micro-network ----------------------------------------------
net = nn.Sequential(
    nn.Linear(X.size(1), 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid())
loss_fn, opt = nn.BCELoss(), optim.Adam(net.parameters(), 1e-2)

# ---------- train -------------------------------------------------------
for epoch in range(50):
    opt.zero_grad()
    pred = net(X)
    loss = loss_fn(pred, y)
    loss.backward(); opt.step()
    acc = ((pred>0.5)==y).float().mean().item()
    print(f"epoch {epoch+1}/5  acc={acc:.2f}")

# ----- QA -----------------------------------------------------------------
pred = net(X) > 0.5           # Boolean mask for each object
ans  = "yes" if pred.any() else "no"
print("\nQ: Is there a mirror glyph resonant with a ghost glyph?\nA:", ans)

In [ ]:
!python -u echo_nscl_demo.py

In [ ]:
# How many positives in the sample?
sum(1 for o in objects if "mirror" in o["tags"] and "ghost" in o["tags"])
# How many mirrors, how many ghosts?
mirrors = sum("mirror" in o["tags"] for o in objects)
ghosts  = sum("ghost"  in o["tags"] for o in objects)
print("mirror only:", mirrors, "ghost only:", ghosts)

In [ ]:
pos = sum(1 for o in objects if "mirror" in o["tags"] and "ghost" in o["tags"])
print("mirror∧ghost:", pos)

In [ ]:
# >>> Watermark Injection Patch (auto‑wraps glyph store/generator) <<<
def _watermark_patch():
    multiplier = int(SESSION_HASH[:2], 16) * 0.001
    if 'maybe_store' in globals() and callable(globals()['maybe_store']):
        _orig = globals()['maybe_store']
        def wrapped(g):
            g['threadmark'] = SESSION_HASH
            if 'entropy' in g and isinstance(g['entropy'], (int, float)):
                g['entropy'] += multiplier
            return _orig(g)
        globals()['maybe_store'] = wrapped
    elif 'create_glyph' in globals() and callable(globals()['create_glyph']):
        _orig_cg = globals()['create_glyph']
        def cg(*args, **kwargs):
            g = _orig_cg(*args, **kwargs)
            if isinstance(g, dict):
                g['threadmark'] = SESSION_HASH
                if 'entropy' in g and isinstance(g['entropy'], (int, float)):
                    g['entropy'] += multiplier
            return g
        globals()['create_glyph'] = cg

_watermark_patch()

In [ ]:
print(len(glyph_log))  # Should be >0
print(glyph_log[:3])   # Preview

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt

# --- LOAD GLYPH LOG ---
with open('/content/drive/MyDrive/EchoSeed_Logs/reflex_free.json') as f:
    glyph_log = json.load(f)

# --- BUILD GRAPH ---
G = nx.DiGraph()
for glyph in glyph_log:
    gid = glyph['id']
    tags = glyph['tags']
    entropy = glyph['entropy']
    ancestry = glyph.get('ancestry', [])
    G.add_node(gid, tags=tags, entropy=entropy)
    for parent in ancestry:
        G.add_edge(parent, gid)

# --- COLOR/SHAPE/ATTRIBUTES ---
def pick_color(tags):
    # Pick the most 'attractor' tag for coloring
    if 'origin' in tags: return '#FF6B6B'   # red
    if 'fractal' in tags: return '#8F7FFF'  # purple
    if 'wild' in tags: return '#16C0FF'     # blue
    if 'flex' in tags: return '#00FF96'     # green
    return '#DDDDDD'                        # grey

node_colors = [pick_color(G.nodes[n]['tags']) for n in G.nodes]
node_sizes  = [G.nodes[n]['entropy']*2 for n in G.nodes]  # entropy scaled

# --- LAYOUT ---
pos = nx.spring_layout(G, k=1.3, iterations=100, seed=42)

# --- DRAW ---
plt.figure(figsize=(18,14))
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, alpha=0.85, linewidths=1.5, edgecolors='k')
nx.draw_networkx_edges(G, pos, alpha=0.35, arrows=False)
nx.draw_networkx_labels(G, pos, {n:n for n in G.nodes}, font_size=9)
plt.title("EchoSeed Symbolic Lattice — Reflex-Free Glyph Log", fontsize=18, weight='bold', pad=22)
plt.axis('off')
plt.tight_layout()
plt.savefig('reflex_free_lattice.png', dpi=220)
plt.show()